In [1]:
import os
import sys
import absl.logging
import PIL.Image

import numpy as np
import pandas as pd
import tensorflow as tf

from typing import Callable
from tensorflow import keras

module_path = os.path.abspath(os.path.join('..', '..', '..'))

if module_path not in sys.path:
    sys.path.append(os.path.join(module_path))

from functions.ciou import ciou_loss, ciou_metric
from functions.loading_data import SMALLER_HEIGHT, SMALLER_WIDTH
from functions.model_running import get_run_number, finalize_run

absl.logging.set_verbosity(absl.logging.ERROR)

In [2]:
get_names = lambda root_path: [
    file_name.split('.')[0]
    for dir_path, _, file_names in os.walk(root_path)
    for file_name in file_names
]
get_paths = lambda path: [f'{os.path.join(root, file)}' for root, dirs, files in os.walk(path) for file in files]
base_dir = os.path.join('..', '..', '..', '..', '..', 'data1', 'images_original_inception_resnet_v2_200x150_splitted')
train_dir = os.path.join(base_dir, 'training')
valid_dir = os.path.join(base_dir, 'validation')

In [3]:
def get_images_array(paths: list[str]) -> np.ndarray:
    rows = []
    rescale = keras.layers.Rescaling(1./255)

    for path in paths:
        with PIL.Image.open(path) as image:
            image_array = np.asarray(image)
            rescaled_image = rescale(image_array)
            grayscale_image = tf.image.rgb_to_grayscale(rescaled_image)

            rows.append(grayscale_image)

    return np.array(rows)


train_paths = get_paths(train_dir)
valid_paths = get_paths(valid_dir)
X_train = get_images_array(train_paths)
X_valid = get_images_array(valid_paths)

In [4]:
train_names = set(get_names(train_dir))
valid_names = set(get_names(valid_dir))
metadata_path = os.path.join('..', '..', '..', '..', '..', 'data1', 'HAM10000_metadata_ext.csv')
data = pd.read_csv(metadata_path).sort_values(by='image_id')
relevant_cols = ['top', 'left', 'bottom', 'right']
train_df = data[data['image_id'].isin(train_names)][relevant_cols]
valid_df = data[data['image_id'].isin(valid_names)][relevant_cols]
ys_train = train_df.to_numpy().astype(float)
ys_train[:, [0, 2]] /= SMALLER_HEIGHT
ys_train[:, [1, 3]] /= SMALLER_WIDTH
ys_valid = valid_df.to_numpy().astype(float)
ys_valid[:, [0, 2]] /= SMALLER_HEIGHT
ys_valid[:, [1, 3]] /= SMALLER_WIDTH

In [5]:
def get_baseline_model(loss: Callable, metric: Callable) -> keras.Model:
    def get_conv_module(prev: keras.layers.Layer, filters: int, kernel_size: int) -> keras.layers.Layer:
        x = keras.layers.Conv2D(filters, kernel_size, padding='same', activation='relu')(prev)
        x = keras.layers.Conv2D(filters * 2, kernel_size, padding='same', activation='relu')(x)
        x = keras.layers.MaxPooling2D()(x)

        return x


    _input = keras.layers.Input(shape=(SMALLER_HEIGHT, SMALLER_WIDTH, 1))
    conv_module = get_conv_module(_input, 16, 7)
    conv_module = get_conv_module(conv_module, 32, 5)
    conv_module = get_conv_module(conv_module, 64, 5)
    conv_module = get_conv_module(conv_module, 128, 3)
    conv_module = get_conv_module(conv_module, 256, 3)
    conv_module = keras.layers.Flatten()(conv_module)
    locator_module = keras.layers.Dense(128, activation='relu')(conv_module)
    locator_module = keras.layers.Dense(4, activation='sigmoid')(locator_module)

    model = keras.Model(_input, locator_module)

    model.compile(optimizer='adam', loss=loss, metrics=[metric])

    print(model.summary())

    return model

In [6]:
def run_model(
        model_factory: Callable,
        model_name: str,
        loss: Callable,
        metric: Callable,
        reduction_patience=5,
        monitor='val_ciou_metric'):
    MIN_DELTA = .001
    early_stopping = keras.callbacks.EarlyStopping(
        monitor=monitor,
        mode='max',
        patience=20,
        min_delta=MIN_DELTA)
    reduce_lr = keras.callbacks.ReduceLROnPlateau(
        monitor=monitor,
        mode='max',
        factor=0.95,
        min_delta=MIN_DELTA,
        patience=reduction_patience,
        min_lr=0.0005,
        verbose=1)
    model_checkpoint = keras.callbacks.ModelCheckpoint(
        filepath=os.path.join('..', '..', '..', '..', 'tmp_models', model_name + '_{epoch}'),
        save_best_only=True)
    tensor_board = keras.callbacks.TensorBoard(log_dir=os.path.join('..', '..', '..', '..', 'tensor_logs', model_name))
    model = model_factory(loss, metric)

    return model.fit(
        X_train,
        ys_train,
        validation_data=(X_valid, ys_valid),
        epochs=500,
        batch_size=64,
        callbacks=[reduce_lr, model_checkpoint, tensor_board, early_stopping])

In [ ]:
model_base_name = 'custom_models_1_starting_point_model'
run_number = get_run_number(model_base_name)
model_name = f'{model_base_name}_{run_number}'
history = run_model(get_baseline_model, model_name, ciou_loss, ciou_metric)
ROOT = os.path.join('..', '..', '..', '..')
DS_NAME = 'data1_roi'
plot_name = f'{model_name}.pdf'

finalize_run(ROOT, plot_name, model_base_name, DS_NAME, history, plot_mode = 'multiple')

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 150, 200, 1)]     0         
                                                                 
 conv2d_10 (Conv2D)          (None, 150, 200, 16)      160       
                                                                 
 conv2d_11 (Conv2D)          (None, 150, 200, 32)      4640      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 75, 100, 32)      0         
 2D)                                                             
                                                                 
 conv2d_12 (Conv2D)          (None, 75, 100, 32)       9248      
                                                                 
 conv2d_13 (Conv2D)          (None, 75, 100, 64)       18496     
                                                           

INFO:tensorflow:Assets written to: ..\..\..\..\tmp_models\custom_models_1_starting_point_model_44_1\assets


126/126 [==============================] - 23s 171ms/step - loss: 0.5290 - ciou_metric: 0.4713 - val_loss: 0.4954 - val_ciou_metric: 0.5043 - lr: 0.0010
Epoch 2/500
125/126 [============================>.] - ETA: 0s - loss: 0.4940 - ciou_metric: 0.5060INFO:tensorflow:Assets written to: ..\..\..\..\tmp_models\custom_models_1_starting_point_model_44_2\assets


INFO:tensorflow:Assets written to: ..\..\..\..\tmp_models\custom_models_1_starting_point_model_44_2\assets


126/126 [==============================] - 21s 170ms/step - loss: 0.4941 - ciou_metric: 0.5057 - val_loss: 0.4949 - val_ciou_metric: 0.5049 - lr: 0.0010
Epoch 3/500
126/126 [==============================] - 20s 155ms/step - loss: 0.4983 - ciou_metric: 0.5015 - val_loss: 0.5045 - val_ciou_metric: 0.4955 - lr: 0.0010
Epoch 4/500
126/126 [==============================] - 20s 155ms/step - loss: 0.4930 - ciou_metric: 0.5070 - val_loss: 0.4959 - val_ciou_metric: 0.5039 - lr: 0.0010
Epoch 5/500
126/126 [==============================] - 20s 156ms/step - loss: 0.4935 - ciou_metric: 0.5062 - val_loss: 0.4964 - val_ciou_metric: 0.5034 - lr: 0.0010
Epoch 6/500
125/126 [============================>.] - ETA: 0s - loss: 0.4935 - ciou_metric: 0.5065
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0009500000451225787.
126/126 [==============================] - 20s 156ms/step - loss: 0.4934 - ciou_metric: 0.5067 - val_loss: 0.4968 - val_ciou_metric: 0.5031 - lr: 0.0010
Epoch 7/500
126/126 [==

INFO:tensorflow:Assets written to: ..\..\..\..\tmp_models\custom_models_1_starting_point_model_44_10\assets


126/126 [==============================] - 21s 168ms/step - loss: 0.4918 - ciou_metric: 0.5088 - val_loss: 0.4946 - val_ciou_metric: 0.5051 - lr: 9.5000e-04
Epoch 11/500
125/126 [============================>.] - ETA: 0s - loss: 0.4915 - ciou_metric: 0.5085
Epoch 11: ReduceLROnPlateau reducing learning rate to 0.0009025000152178108.
126/126 [==============================] - 19s 154ms/step - loss: 0.4915 - ciou_metric: 0.5085 - val_loss: 0.4986 - val_ciou_metric: 0.5012 - lr: 9.5000e-04
Epoch 12/500
126/126 [==============================] - 19s 154ms/step - loss: 0.4912 - ciou_metric: 0.5088 - val_loss: 0.4956 - val_ciou_metric: 0.5040 - lr: 9.0250e-04
Epoch 13/500
 73/126 [================>.............] - ETA: 7s - loss: 0.4902 - ciou_metric: 0.5098

In [12]:
models_dir = os.path.join(
    '..',
    '..',
    '..',
    '..',
    'models',
    'data1_roi')
model_path = os.path.join(models_dir, 'custom_models_1_starting_point_model_42_3')
model = keras.models.load_model(model_path, compile=False, custom_objects={'loss': ciou_loss, 'ciou_metric': ciou_metric})

model.compile(optimizer='adam', loss=ciou_loss, metrics=[ciou_metric])

results = model.predict(X_valid)

63/63 [==============================] - 2s 19ms/step


In [13]:
print(results)

[[0.1445083  0.2022213  0.8415673  0.7830499 ]
 [0.14451341 0.20222144 0.8415655  0.7830422 ]
 [0.14449254 0.20219348 0.84158355 0.78306454]
 ...
 [0.14450324 0.20221291 0.8415717  0.7830535 ]
 [0.14448054 0.20218377 0.8415976  0.7830799 ]
 [0.14445537 0.20215933 0.8416297  0.7830942 ]]


In [19]:
ys_train[:, 0].mean()

0.1840091495113329

In [20]:
ys_train[:, 1].mean()

0.22492451653150342

In [21]:
ys_train[:, 2].mean()

0.8106417134539405

In [22]:
ys_train[:, 3].mean()

0.7599744229569557